In [22]:
%reset -f
%matplotlib qt

import pandas as pd
import math
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt

##################################################
## Comparison of training sets for -214 and -251N
##
## Comparing the thrust characteristics of the CEO
## and NEO (1700 fields vs. 20,000 fields total)
##
##################################################
## Author: J.M. Roldan
## Date: 11/21/2020
## License: GPL 3.0
## Discord: TazX [Z+1]#0405
## Status: Final
##################################################

################################
# Training and Testing Functions
################################
def regression(df_214, X, y, order):
    
    #generate a model of polynomial features
    poly = preprocessing.PolynomialFeatures(degree=order)

    #transform the x data for proper fitting (for single variable type it returns,[1,x,x**2])
    X_poly = poly.fit_transform(X)

    #generate the regression object
    regr = linear_model.LinearRegression()

    #preform the actual regression
    regr.fit(X_poly, y)

    #some statistics
    r_sq = regr.score(X_poly,y)
    b0 = regr.intercept_

    #new DF and check with original table
    columns = ['Mach', 'Alt', 'N1', 'FN_orig', 'FN_pred', 'error']
    new_df = pd.DataFrame(columns=columns)
    m = regr.coef_

    ####################
    # prediction loop
    ####################
    for index in range(len(df_214.index)):
        Mach = df_214['TAS'][index]
        Alt = df_214['Alt'][index]
        N1 = df_214['N1'][index]
        WEIGHT = df_214['WEIGHT'][index]
        
        predict_ = poly.fit_transform(([[Mach, Alt]]))
        predFN = regr.predict(predict_)
        predFN = predFN[0] 

        sq_error = (df_214['FN100'][index]-predFN)**2
        new_df = new_df.append({'Mach':Mach,\
                                'Alt':Alt,\
                                'N1':N1,\
                                'WEIGHT':WEIGHT,\
                                'FN_orig':df_214['FN100'][index],\
                                'FN_pred':predFN,\
                                'error':sq_error}, ignore_index = True)

    mse = math.sqrt(sum(new_df['error'])/(len(new_df.index)-2))
    r_sq*=100
    print("Training --- Order: %d   R2 score: %.5f %%   Std Error: %.5f" % (order, r_sq, mse))
    
    return poly, regr, new_df


def testing(df_251, poly, regr):
    columns = ['TAS', 'Alt', 'N1', 'Weight', 'PCFN', 'FN_pred', 'error']
    new_df = pd.DataFrame(columns=columns)

    for index in range(len(df_251.index)):
            Mach = df_251['TAS'][index]
            Alt = df_251['Alt'][index]
            N1 = df_251['N1'][index]
            WEIGHT = df_251['WEIGHT'][index]
            PCFN = df_251['PCFN'][index]
            
            predict_ = poly.fit_transform(([[Mach, Alt]]))
            predFN = regr.predict(predict_)
            predFN = predFN[0]

            #predFN = b0+m[0]+(m[1]*Mach)+(m[2]*Alt)+(m[3]*(Mach**2))+(m[4]*Mach*Alt)+\
                      #(m[5]*(Alt**2))+(m[6]*(Mach**3))+(m[7]*(Mach**2)*Alt)+(m[8]*Mach*(Alt**2))+(m[9]*(Alt**3))

            sq_error = (df_251['FN100'][index]-predFN)**2
            new_df = new_df.append({'TAS':Mach,\
                                    'Alt':Alt,\
                                    'N1':N1,\
                                    'WEIGHT':WEIGHT,\
                                    'PCFN':PCFN,\
                                    'FN_pred':predFN,\
                                    'error':sq_error}, ignore_index = True)

   
    return new_df

################
# Figure Set-up
################

a4_dims = (11.7, 8.27)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('Mach')
ax.set_ylabel('Altitude')
ax.set_zlabel('FN')

################
# Data Init
################
train = 0
phase = 'TO'
order = 3
    
# CSV read and tiddy-up
df_214 = pd.read_csv("a320-214.csv", low_memory=False)
df_214 = df_214.rename(columns = {'ALT.':'Alt', 'MACH':'Mach'}, inplace=False)

df_214 = df_214[df_214['PHASE'] == phase]
df_214 = df_214.reset_index(drop=True)

df_251 = pd.read_csv("a320-251n.csv", low_memory=False)
df_251 = df_251.rename(columns = {'ALT.':'Alt', 'MACH':'Mach'}, inplace=False)

df_251 = df_251[df_251['PHASE'] == phase]
df_251 = df_251.reset_index(drop=True)

##############################################################
# Main Routine
##############################################################

# Define Independent and Dependent Variables
X = df_214[['TAS', 'Alt']]
y = df_214['FN']

if (train == 1):
    for i in range(1,12):
        regression(df_214, X, y, i)
else:
    poly, regr, new_df = regression(df_214, X, y, order)
    new_df = testing(df_251, poly, regr)

    new_df['FN_pred'] = new_df['FN_pred']*1.05
    ax.plot(df_214['TAS'], df_214['Alt'], df_214['FN'], markersize="2", linestyle="none", marker=".", mfc="none", markeredgecolor="red")
    ax.plot(new_df['TAS'], new_df['Alt'], new_df['FN_pred'], markersize="2", linestyle="none", marker=".", mfc="none", markeredgecolor="blue")
#         file = "a320-251_" + str(altitude) + ".csv"
#         new_df.to_csv(file, index = False, header=True)
    

Training --- Order: 3   R2 score: 97.56104 %   Std Error: 1185.36339
